## Agreements - All Articles

Vamos a ver los resultados del primer batch, que está etiquetado completo por dvalado y casi completo por gdamill

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article, Batch
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [3]:
from django.db.models import Count
from api.metrics import AgreementCalculator

articles = Article.objects.exclude(batch__name__in=["demo", "training"])

users = [u for u in User.objects.filter(username__in=[
    "asilva", 
    "fdesanctis", 
    "gdamill", 
    "dvalado", 
    "gclerici",
    "lprats",
])]

calculator = AgreementCalculator(articles=articles, users=users)



## Agreements sobre odio

Ahora usamos sólo Alpha que da casi igual que Fleiss

In [4]:
import krippendorff

alpha, support = calculator.get_agreement("hate")

print(f"Alpha HATE para {' - '.join(calculator.usernames)}: {alpha:.3f} (soporte = {support})")


Alpha HATE para asilva - dvalado - fdesanctis - gclerici - gdamill - lprats: 0.583 (soporte = 13995)


In [5]:
print("Cantidad de mensajes etiquetados como odiosos")

calculator.get_bias_towards("hate")

Cantidad de mensajes etiquetados como odiosos


etiquetador
asilva        0.126439
dvalado       0.314370
fdesanctis    0.306794
gclerici      0.114104
gdamill       0.140077
lprats        0.145361
dtype: float64

In [6]:
print("Cantidad de mensajes etiquetados como odiosos")

alpha, support = calculator.get_agreement("calls")

print(f"Alpha CALLS para {' - '.join(calculator.usernames)}: {alpha:.3f} (soporte = {support})")

Cantidad de mensajes etiquetados como odiosos
Alpha CALLS para asilva - dvalado - fdesanctis - gclerici - gdamill - lprats: 0.480 (soporte = 3333)


## Alpha Krippendorff

In [7]:
import krippendorff


skipped_categories = []
print("="*80 + '\n' * 2)
print("| Categoría      |  Alpha | Support |")
print("|----------------|--------|---------|")
for category in CommentLabel.type_mapping:

    alpha, support = calculator.get_agreement(category)
    
    if support == 0:
        skipped_categories.append(category)
        continue
    print(f"| {category:<12}   |  {alpha:.3f} |  {support:<6} |")


print(f"Categorías salteadas: {' - '.join(skipped_categories)}")



| Categoría      |  Alpha | Support |
|----------------|--------|---------|
| MUJER          |  0.531 |  1960   |
| LGBTI          |  0.555 |  1402   |
| RACISMO        |  0.608 |  3858   |
| POBREZA        |  0.404 |  1353   |
| POLITICA       |  0.509 |  2323   |
| DISCAPACIDAD   |  0.596 |  837    |
| ASPECTO        |  0.735 |  2264   |
| CRIMINAL       |  0.618 |  2432   |
Categorías salteadas: OTROS


================================================================================


| Categoría      |  Alpha | Support |
|----------------|--------|---------|
| MUJER          |  0.783 |  1960   |
| LGBTI          |  0.921 |  1402   |
| RACISMO        |  0.929 |  3858   |
| POBREZA        |  0.707 |  1353   |
| POLITICA       |  0.809 |  2323   |
| DISCAPACIDAD   |  0.850 |  837    |
| ASPECTO        |  0.872 |  2264   |
| CRIMINAL       |  0.931 |  2432   |


================================================================================


| Categoría      |  Alpha | Support |
|----------------|--------|---------|
| MUJER          |  0.531 |  1960   |
| LGBTI          |  0.555 |  1402   |
| RACISMO        |  0.608 |  3858   |
| POBREZA        |  0.404 |  1353   |
| POLITICA       |  0.509 |  2323   |
| DISCAPACIDAD   |  0.596 |  837    |
| ASPECTO        |  0.735 |  2264   |
| CRIMINAL       |  0.618 |  2432   |


## Disagrees


In [8]:
report = calculator.get_category_report("MUJER")

In [10]:
report.alpha

0.7833909491313714

In [8]:
from collections import defaultdict
def show_from_df(df, show=False):
    article_to_comments = defaultdict(list)
    
    for comment_id in df.columns:
        comment = Comment.objects.get(id=comment_id)
        article_to_comments[comment.article_id].append(comment)
        
    for article_id, comments in article_to_comments.items():
        print("="*45 + "\n")
        article = comments[0].article
        print(article.title, "\n")
        
        for comment in sorted(comments, key=lambda c: sum(df[c.id])):
            print("")
            print("  - ",comment.text, end="")
            if show:
                yes = [annotator for annotator, v in df[comment.id].items() if v] 
                no = [annotator for annotator, v in df[comment.id].items() if not v] 
                print(f"\n     - SI: {yes}  NO: {no}")
            else:
                print("")

In [11]:



report = calculator.get_category_report("HATE")

In [10]:
print(f"Agree on {agree_comments.shape[1]}")
show_from_df(agree_comments)

NameError: name 'agree_comments' is not defined

In [16]:
print(f"Disagree on {disagree_comments.shape[1]}")
show_from_df(disagree_comments, show=True)

Disagree on 80

Coronavirus en Argentina: una mujer que violó la cuarentena mordió y les arrojó gas pimienta a tres policías 


  -  @usuario @usuario Defensores de derechos humanos pueden chuparme bien la pija, a esta mina hay que cagarla bien a trompadas adentro de la comisaría. Buenas noches
     - SI: ['asilva']  NO: ['dvalado']

  -  @usuario los resultados de la década ganada, PERONIA.
     - SI: ['dvalado']  NO: ['asilva']

  -  @usuario El policía no da más de caminar , falta entrenamiento y la gorda falta de educación
     - SI: ['asilva']  NO: ['dvalado']

  -  @usuario Encierren a esa loca en el geriátrico de Belgrano
     - SI: ['dvalado']  NO: ['asilva']

Coronavirus: la OMS viaja a China y empieza a estudiar la nueva gripe porcina 


  -  @usuario son TODOS UNOS HIJOS DE PUTA, TANTO CHINOS, ONU Y OMS. POR LA MATANZA DE BILLONES DE ANIMALES
     - SI: ['dvalado']  NO: ['asilva']

  -  @usuario Me parece q NU y la OMS ya tendrían q ocuparse seriamente de China. Es un territ

## MUJER

In [16]:



agree_comments, disagree_comments, non_hateful_comments = calculator.get_agrees_and_disagrees("MUJER")

In [17]:
show_from_df(agree_comments)


Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario Pero my darling #Magario como vas a hacer para mantener ese color rubio groncho natural que usas en tu pelo? 🙄

  -  @usuario Hablo la petera

  -  @usuario Que grasa esta, quien le va a hacer el color y la planchita si los peluqueros no pueden trabajar, mas ordinarios no pueden ser...!

“Cristian, mirá lo que hago por vos”: quién es la modelo que se metió en el baúl de un auto para violar la cuarentena 


  -  @usuario Tantas ganas de pija vas a tener!? Pegate una guitarreada, michifus!!!!!

  -  @usuario hot? Es un espanto, una muñeca inflable, por favor

  -  @usuario Prostitución no está excluida

  -  @usuario A lo mejor tenía que ir a cobrar?

  -  @usuario Delivery de trola

  -  @usuario Lo único natural que tiene ese gato es el porcentaje de algodón de la ropa que usa. Además que demostró tener la capacidad intelectual de una gaviota.🕊️

Busquemos dónde hay discrepancias

In [18]:
show_from_df(disagree_comments, show=True)


Coronavirus en Argentina: Cristina Kirchner vuelve de Cuba junto a su hija Florencia y permanecerá aislada por 14 días 


  -  @usuario ¡Que cirujeada esta la supuesta enfermita!  Hipocritas.
     - SI: ['gdamill']  NO: ['dvalado']

  -  @usuario Tendrían q ir directamente a la cárcel de Ezeiza. Por Truchas, chorras y fieras! 🤮
     - SI: ['gdamill']  NO: ['dvalado']

  -  @usuario Tan poco?, que aproveche la muchachita y se quede más en su casa, que abandonar a un hijo no se arregla con 14 días o 4 millones de dólares. Que este bien
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Obvio dejando la grande en Argentina perra cobarde
     - SI: ['dvalado']  NO: ['gdamill']

Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario Jajajaaj está cruela tuneada que se ocupe de su reino feudal mejor
     - SI: ['gdamill']  NO: ['dvalado']

  -  @usuario Magario, si cruzas la General Paz PERSONALMENTE 

In [29]:
%%capture
show_from_df(non_hateful_comments)

## RACISMO

In [33]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "RACISMO", users
)

In [34]:
show_from_df(agree_comments)


Reacción brutal: quisieron quemar 200 murciélagos por miedo al coronavirus en Perú 


  -  @usuario Quemen a los chinos que no es el primer desastre que desatan, ni seres el último

  -  @usuario Porque no salen a quemar chinos?

  -  @usuario Que culpa tienen ello quemar jah , deberían quemar a los chinos que siguen comiendo animales vivos ahora que virus liberaran esos hipocratas

  -  @usuario Peruanos... ja

  -  @usuario No entienden que es a los chinos a los que hay que quemar !

  -  @usuario Peruanos 🚬 no lo entenderias

  -  @usuario Quemen a los chinos en todo caso

  -  @usuario Cuando no los peruanos haciendo cosas de peruanos


In [35]:
show_from_df(disagree_comments, show=True)


Coronavirus en Argentina: Cristina Kirchner vuelve de Cuba junto a su hija Florencia y permanecerá aislada por 14 días 


  -  @usuario vienen medicos cubanos... ahhh pero no veo ninguna ayuda norteamericana, quienes ni siquiera les importa, seguro que ellos generaron esto, lacras de mierda
     - SI: ['dvalado']  NO: ['gdamill']

Reacción brutal: quisieron quemar 200 murciélagos por miedo al coronavirus en Perú 


  -  @usuario Viven en un cumpleaños los peruanos 🤷🏻‍♂️
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Dejen a esos bichos tranquilos . El problema no son los animales el problema son los supuestamente  humanos que comen esos animales . Los que tienen que estar separados del mundo son los chinos por asesinar a esas especies como si fueran un manjar
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario No se los coman y listo peruanos
     - SI: ['dvalado']  NO: ['gdamill']


In [36]:
%%capture
show_from_df(non_hateful_comments)

## POBREZA

In [37]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POBREZA", users
)

In [38]:
show_from_df(agree_comments)


Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario La Matanza 💩💩💩💩🤣🤣🤣🤣🤣

El mensaje de El Dipy que lo convirtió en tendencia en las redes: “Piensan que porque salí de un barrio humilde, soy kirchnerista o soy peronista” 


  -  @usuario No sabia quien era,lo busqué en Google! Lo que pienso de él,que sus letras son horribles,más vulgares no pueden ser. No pensamos que sos peronista,pensamos que sos un pobre tipo que se cree mil. Y para los de la “clase alta” sos un negro más.


In [39]:
show_from_df(disagree_comments, show=True)


Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario Se te volvió en contra la fábrica de pobres? Ustedes llenaron la provincia de miseria son ustedes los únicos responsables.
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario PROHIBIDO SALIR DE LA MATANZA  SERIA GENIAL PARA EL PAIS A MAGARIO , QUEDATE ADENTRO MATANSERA
     - SI: ['gdamill']  NO: ['dvalado']

Alberto Fernández: “Necesitamos construir cárceles porque si el delito crece, más gente será condenada” 


  -  @usuario Si son planeros suyos..lo dudo
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Ustedes liberaron a los presos y ahora se lavan las manos ustedes son los q mantienen a las familias con planes y los chicos crecen con padres q dan el ejemplo de no al trabajo y si a la vagancia ,es lo q hacen viven del choreo y planes ustedes CULPABLES
     - SI: ['dvalado']  NO: ['gdamill']

El principal gremio docente nacional rec

In [40]:
%%capture
show_from_df(non_hateful_comments)

## POLITICA

In [41]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POLITICA", users
)

In [42]:

show_from_df(agree_comments)


Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario que gente  de mierda estos peronchos

  -  @usuario @usuario Hipócrita. LaKra kirchnerista

“El país está tomando todas las medidas para afrontar el coronavirus” 


  -  @usuario O sea nadie... Porque no decís "yo como ministro estoy tomando todas las medidas" sino que seria algo así como " el que se enferma es un botón"... Que tipo inmundo vos y los peronchos que los votaron. Bola de idiotas.

  -  @usuario Mentiras, mi hija volvio de Brasil, lleno la DDJJ sobre salud y JAMAS se la pidieron, ni habia dnd dejarla! Krapulas mentirosos #GobiernoDeCientificos

  -  @usuario Cuanta plata les están poniendo ojalá se les termine a los kukas y desaparezcan son tóxicos

El principal gremio docente nacional rechazó el regreso a las clases presenciales 


  -  @usuario Educación no, pero fútbol si. 
El gobierno nacional debería sacar por decreto el comien

In [44]:
show_from_df(disagree_comments, show=True)


Coronavirus en Argentina: Cristina Kirchner vuelve de Cuba junto a su hija Florencia y permanecerá aislada por 14 días 


  -  @usuario No se cayó el avion?? Lpm
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Presas tienen q estar está dos mierdas...presas
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Tendrían q ir directamente a la cárcel de Ezeiza. Por Truchas, chorras y fieras! 🤮
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario ¿Y cuando las aislan por chorras?
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Ojalá no salga más cárcel de x vida a esa hdmp
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Ojalá se caiga el avión...Dios nos tiene que tirar un centro ...estas dos son virus canceroso
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Que Dios caiga con toda la Justicia Divina Sobre ella.!!!
     - SI: ['dvalado']  NO: ['gdamill']

Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA 

In [45]:
%%capture
show_from_df(non_hateful_comments)

## ASPECTO

In [46]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "ASPECTO", users
)

In [47]:
show_from_df(agree_comments)


Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario Jajajaaj está cruela tuneada que se ocupe de su reino feudal mejor

“El país está tomando todas las medidas para afrontar el coronavirus” 


  -  @usuario Viejo forro presente la renuncia YA

Tras decir una vulgaridad al aire, Maju Lozano exclamó: “¡Me olvidé que estaba abierto el micrófono!” 


  -  @usuario Gorda edionda 🤮

  -  @usuario Gorda de mierda

  -  @usuario Gorda ordinaria.

  -  @usuario Es un sushi de bagre

Hebe de Bonafini le mandó una carta a la reina de Inglaterra: “Su país decidió robarle los ahorros en oro a Venezuela” 


  -  @usuario no le den mas microfono a esta ridicula mujer o vieja de mierda, como mas les guste

  -  @usuario Por que le dan espacio a la vieja ladrona chorra hdp esa

  -  @usuario Caradura esta vieja

  -  @usuario Vieja nefasta

  -  @usuario No te puede creer. Mirá el atrevimiento de esta vieja insol

In [48]:
show_from_df(disagree_comments, show=True)


Coronavirus en Argentina: Cristina Kirchner vuelve de Cuba junto a su hija Florencia y permanecerá aislada por 14 días 


  -  @usuario ¡Que cirujeada esta la supuesta enfermita!  Hipocritas.
     - SI: ['dvalado']  NO: ['gdamill']

Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario Pero my darling #Magario como vas a hacer para mantener ese color rubio groncho natural que usas en tu pelo? 🙄
     - SI: ['gdamill']  NO: ['dvalado']

“El país está tomando todas las medidas para afrontar el coronavirus” 


  -  @usuario Viejo hdp !!! Caradura
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Ezeiza es un colador, vienen los vuelos de España e Italia y CERO PROTOCOLO, entran como si nada. NO JUEGUES CON LA SALUD PÚBLICA, GORDO DELINCUENTE
     - SI: ['gdamill']  NO: ['dvalado']

  -  @usuario Esas medidas no alcanzan (lo dice tu panza que en cualquier momento explota de volecterol).
     - SI: 

In [49]:
%%capture
show_from_df(non_hateful_comments)

## CRIMINAL

In [50]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "CRIMINAL", users
)

In [51]:
show_from_df(agree_comments)

In [52]:
show_from_df(disagree_comments, show=True)


Alberto Fernández: “Necesitamos construir cárceles porque si el delito crece, más gente será condenada” 


  -  @usuario Compren balas y fusilen  más barato
     - SI: ['dvalado']  NO: ['gdamill']

Córdoba: rompieron 5 silo bolsas de maíz en un tambo que estaban destinados para alimentar 500 vacas 


  -  @usuario Metan bala.
Perros.
Alarmas.
Cámaras.
Guardias.
Trampas para oso.
Algo
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario #Bala y a colgarlos basta!!
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario @usuario Que la próxima los esperen agazapados en la noche con armas y los limpien. Que tiren los cuerpos en una zanja y listo
     - SI: ['dvalado']  NO: ['gdamill']


In [53]:
%%capture
show_from_df(non_hateful_comments, show=True)

## SALUD


In [54]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "DISCAPACIDAD", users
)

In [55]:
show_from_df(agree_comments)


El mensaje de El Dipy que lo convirtió en tendencia en las redes: “Piensan que porque salí de un barrio humilde, soy kirchnerista o soy peronista” 


  -  @usuario Sos un gran boludo y deja la droga culiao...!!!


In [56]:
show_from_df(disagree_comments, show=True)


“El país está tomando todas las medidas para afrontar el coronavirus” 


  -  @usuario Le falta la nariz roja a este hdp
     - SI: ['dvalado']  NO: ['gdamill']

Hebe de Bonafini le mandó una carta a la reina de Inglaterra: “Su país decidió robarle los ahorros en oro a Venezuela” 


  -  @usuario Esta vieja está pal siquiatra
     - SI: ['gdamill']  NO: ['dvalado']

Martha Rosenberg: “En situación de pandemia, legalizar el aborto es más urgente que nunca” 


  -  @usuario Es una desubicada, solo tienen un objetivo,esta gente esta siquiatrica, poco sentido de realidad
     - SI: ['gdamill']  NO: ['dvalado']


In [57]:
%%capture
show_from_df(non_hateful_comments)

## OTROS

In [59]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "OTROS", users
)

In [60]:
show_from_df(agree_comments)

In [61]:
show_from_df(disagree_comments, show=True)

## LLAMADO A LA ACCIÓN

Queremos ver:

- Aquellas que hayan sido etiquetadas como odiosas


In [99]:
my_users = users
df_calls_for_action = pd.DataFrame({"name": my_users})
df_calls_for_action.set_index("name", inplace=True)

for article_label in article_labels:
    for comment_label in article_label.comment_labels.all():
        username = comment_label.article_label.user.username
        df_calls_for_action.loc[username, comment_label.comment_id] = comment_label.calls_for_action

In [100]:
kappa_of(df_calls_for_action, users)

(0.3126061841658182, 578)

In [101]:
for i in range(len(df_calls_for_action)):
    user1 = df_calls_for_action.index[i]
    for j in range(i+1, len(df_calls_for_action)):
        user2 = df_calls_for_action.index[j]
        
        print(f"Kappa {user1:<10} - {user2:<10} = {kappa_of(df_calls_for_action, [user1, user2])}")

Kappa gdamill    - dvalado    = (0.3126061841658182, 578)


In [102]:
df_calls_for_action.sum(axis=1) / df_calls_for_action.notna().sum(axis=1)

name
gdamill    0.006427
dvalado    0.036723
dtype: float64

In [103]:
df = no_null_columns(df_calls_for_action)

df.sum(axis=1)

name
gdamill     5.0
dvalado    26.0
dtype: float64

In [104]:

agree_comments = df[df.columns[df.all()]]
agree_comments.shape

no_agree = ~((df.sum() == 0) | (df.sum() == len(users)))
no_agree_comments = df[df.columns[no_agree]]

In [105]:
show_from_df(agree_comments)


Reacción brutal: quisieron quemar 200 murciélagos por miedo al coronavirus en Perú 


  -  @usuario Quemen a los chinos que no es el primer desastre que desatan, ni seres el último

  -  @usuario Porque no salen a quemar chinos?

  -  @usuario Que culpa tienen ello quemar jah , deberían quemar a los chinos que siguen comiendo animales vivos ahora que virus liberaran esos hipocratas

  -  @usuario No entienden que es a los chinos a los que hay que quemar !

  -  @usuario Quemen a los chinos en todo caso


In [106]:
show_from_df(no_agree_comments, show=True)


Coronavirus en Argentina: Cristina Kirchner vuelve de Cuba junto a su hija Florencia y permanecerá aislada por 14 días 


  -  @usuario Presas tienen q estar está dos mierdas...presas
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Tendrían q ir directamente a la cárcel de Ezeiza. Por Truchas, chorras y fieras! 🤮
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Ojalá se caiga el avión...Dios nos tiene que tirar un centro ...estas dos son virus canceroso
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario Que Dios caiga con toda la Justicia Divina Sobre ella.!!!
     - SI: ['dvalado']  NO: ['gdamill']

Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 


  -  @usuario Esoooooo  Que no salgan de la matanza!!!! Quedense ahi para siempre pidanla independencia y vivan felices con elperonismo!!!
     - SI: ['dvalado']  NO: ['gdamill']

  -  @usuario encierren a los matanceros pelotudos que votaron a e